# XGBoost on NN Residuals - Three-Stage Pipeline

This notebook implements the final stage of the residual modeling pipeline:
1. Linear Regression (exp_005) - captures linear patterns
2. Neural Network on LR residuals (exp_006) - captures non-linear patterns  
3. XGBoost on NN residuals (exp_007) - captures remaining patterns

Expected to improve upon the 0.02047 baseline from direct XGBoost modeling.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
SEED = 42
np.random.seed(SEED)

## Load Data

In [2]:
# Load training data
train_df = pd.read_csv('/home/nonroot/snapshots/playground-series-s5e5/21025419367/code/data/train.csv')
test_df = pd.read_csv('/home/nonroot/snapshots/playground-series-s5e5/21025419367/code/data/test.csv')

# Load residuals from NN stage (exp_006)
residuals_nn = pd.read_csv('/home/code/experiments/006_neural_network_residuals/residuals_after_nn.csv')
print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Residuals from NN shape: {residuals_nn.shape}")
print(f"\nResiduals statistics:")
print(residuals_nn['residual'].describe())

Training data shape: (8000, 9)
Test data shape: (2000, 9)
Residuals from NN shape: (8000, 2)

Residuals statistics:
count    8000.000000
mean       -0.050198
std        20.603560
min       -81.094214
25%       -13.991381
50%         0.005975
75%        13.958964
max        74.479004
Name: residual, dtype: float64


## Feature Engineering

Create features for XGBoost stage. Based on winning solutions analysis:
- Log1p transforms of numeric features
- Product features (pairwise interactions)
- Binned features for categorical handling
- Groupby z-score features

In [3]:
def engineer_features(df):
    """Engineer features for XGBoost on residuals"""
    df = df.copy()
    
    # Original numeric features
    numeric_features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
    
    # 1. Log1p transforms
    for col in numeric_features:
        df[f'{col}_log1p'] = np.log1p(df[col])
    
    # 2. Product features (pairwise interactions) - these are valuable but caused overfitting in direct modeling
    # Now applied to residuals where they may capture remaining patterns
    df['product_Weight_Duration'] = df['Weight'] * df['Duration']
    df['product_Duration_Heart_Rate'] = df['Duration'] * df['Heart_Rate']
    df['product_Height_Weight'] = df['Height'] * df['Weight']
    df['product_Age_Weight'] = df['Age'] * df['Weight']
    df['product_Duration_Body_Temp'] = df['Duration'] * df['Body_Temp']
    
    # 3. Ratio features
    df['ratio_Weight_Height'] = df['Weight'] / (df['Height'] + 1e-6)
    df['ratio_BMI'] = df['Weight'] / ((df['Height'] / 100) ** 2 + 1e-6)
    df['ratio_Duration_Weight'] = df['Duration'] / (df['Weight'] + 1e-6)
    df['ratio_Heart_Rate_Age'] = df['Heart_Rate'] / (df['Age'] + 1e-6)
    
    # 4. Sum features
    df['sum_Weight_Duration'] = df['Weight'] + df['Duration']
    df['sum_Height_Weight'] = df['Height'] + df['Weight']
    df['sum_Age_Weight'] = df['Age'] + df['Weight']
    
    # 5. Difference features
    df['diff_Height_Weight'] = df['Height'] - df['Weight']
    df['diff_Age_Weight'] = df['Age'] - df['Weight']
    df['diff_Duration_Heart_Rate'] = df['Duration'] - df['Heart_Rate']
    
    # 6. Binned features (equal-width bins)
    for col in numeric_features:
        df[f'{col}_binned'] = pd.cut(df[col], bins=9, labels=False)
    
    # 7. One-hot encode Sex and drop original
    if 'Sex' in df.columns:
        sex_dummies = pd.get_dummies(df['Sex'], prefix='Sex')
        df = pd.concat([df, sex_dummies], axis=1)
        df = df.drop(columns=['Sex'])  # Drop original Sex column
    
    return df

# Apply feature engineering to train and test data
print("Engineering features for training data...")
train_features = engineer_features(train_df)
print(f"Train features shape: {train_features.shape}")

print("Engineering features for test data...")
test_features = engineer_features(test_df)
print(f"Test features shape: {test_features.shape}")

# Define feature columns (exclude id and target)
feature_cols = [col for col in train_features.columns if col not in ['id', 'Calories']]
print(f"Number of features: {len(feature_cols)}")
print(f"Feature columns: {feature_cols[:10]}...")  # Show first 10

Engineering features for training data...
Train features shape: (8000, 37)
Engineering features for test data...
Test features shape: (2000, 37)
Number of features: 35
Feature columns: ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp', 'Age_log1p', 'Height_log1p', 'Weight_log1p', 'Duration_log1p']...


## Prepare Data for XGBoost

Target: residuals from NN stage (remaining patterns after Linear+NN)
Features: engineered features above

In [4]:
# Target is residuals from NN stage
y_residuals = residuals_nn['residual'].values

# Features for XGBoost
X_train = train_features[feature_cols]
X_test = test_features[feature_cols]

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_residuals shape: {y_residuals.shape}")
print(f"y_residuals range: [{y_residuals.min():.2f}, {y_residuals.max():.2f}]")

X_train shape: (8000, 35)
X_test shape: (2000, 35)
y_residuals shape: (8000,)
y_residuals range: [-81.09, 74.48]


## Cross-Validation Setup

Use 5-fold CV with same seed as previous stages to ensure consistency.

In [5]:
# 5-fold CV
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

# Initialize arrays for OOF predictions
oof_predictions = np.zeros(len(X_train))
test_predictions = np.zeros(len(X_test))

# Store fold scores
fold_scores = []

print(f"Starting {n_splits}-fold CV...")

Starting 5-fold CV...


## Load OOF Predictions from Previous Stages

Need Linear Regression and Neural Network OOF predictions to combine with XGBoost predictions.

In [6]:
# Load OOF predictions from Linear Regression (exp_005)
lr_oof = pd.read_csv('/home/code/experiments/005_linear_regression/oof_005_linear_regression.csv')
lr_predictions = lr_oof['oof_prediction'].values

# Load OOF predictions from Neural Network (exp_006)  
nn_oof = pd.read_csv('/home/code/experiments/006_neural_network_residuals/oof_006_neural_network_residuals.csv')
nn_residual_predictions = nn_oof['residual_prediction'].values

print(f"LR OOF shape: {lr_predictions.shape}")
print(f"NN residual OOF shape: {nn_residual_predictions.shape}")
print(f"LR predictions range: [{lr_predictions.min():.2f}, {lr_predictions.max():.2f}]")
print(f"NN residual predictions range: [{nn_residual_predictions.min():.2f}, {nn_residual_predictions.max():.2f}]")

LR OOF shape: (8000,)
NN residual OOF shape: (8000,)
LR predictions range: [42.76, 429.20]
NN residual predictions range: [-52.15, 57.38]


## Train XGBoost on Residuals

Train XGBoost to capture remaining patterns in residuals after Linear+NN.

In [7]:
# XGBoost parameters (conservative, as recommended in strategy)
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 5,
    'learning_rate': 0.03,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'n_estimators': 800,
    'random_state': SEED,
    'n_jobs': 4
}

print("Training XGBoost on residuals...")
print(f"Parameters: {params}")

fold = 1
for train_idx, val_idx in kf.split(X_train):
    print(f"\nFold {fold}/{n_splits}")
    
    # Split data
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_residuals[train_idx], y_residuals[val_idx]
    
    # Train XGBoost model
    model = xgb.XGBRegressor(**params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    
    # Predict on validation set (residuals)
    val_residual_pred = model.predict(X_val)
    oof_predictions[val_idx] = val_residual_pred
    
    # Predict on test set
    test_fold_pred = model.predict(X_test)
    test_predictions += test_fold_pred / n_splits
    
    # Calculate fold score (RMSLE on combined predictions)
    # Combined prediction = LR + NN residual + XGB residual
    val_combined_pred = (lr_predictions[val_idx] + 
                        nn_residual_predictions[val_idx] + 
                        val_residual_pred)
    
    # Clip predictions to avoid log(negative)
    val_combined_pred = np.clip(val_combined_pred, 1e-6, None)
    
    fold_score = np.sqrt(mean_squared_log_error(train_df['Calories'].iloc[val_idx], val_combined_pred))
    fold_scores.append(fold_score)
    
    print(f"  Fold {fold} RMSLE: {fold_score:.6f}")
    
    fold += 1

print(f"\n{'='*50}")
print(f"CV RMSLE: {np.mean(fold_scores):.6f} ± {np.std(fold_scores):.6f}")
print(f"Fold scores: {fold_scores}")

Training XGBoost on residuals...
Parameters: {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'max_depth': 5, 'learning_rate': 0.03, 'subsample': 0.8, 'colsample_bytree': 0.8, 'n_estimators': 800, 'random_state': 42, 'n_jobs': 4}

Fold 1/5


  Fold 1 RMSLE: 0.207970

Fold 2/5


  Fold 2 RMSLE: 0.199187

Fold 3/5


  Fold 3 RMSLE: 0.212461

Fold 4/5


  Fold 4 RMSLE: 0.220010

Fold 5/5


  Fold 5 RMSLE: 0.199985

CV RMSLE: 0.207923 ± 0.007823
Fold scores: [0.2079695970354534, 0.19918747729955139, 0.21246057368225243, 0.22001000190857645, 0.1999854477957768]


## Analyze Results

Compare performance to baseline and previous stages.

In [10]:
# Calculate combined OOF predictions
oof_combined = lr_predictions + nn_residual_predictions + oof_predictions

# Clip to avoid log issues
oof_combined = np.clip(oof_combined, 1e-6, None)

# Overall CV score
cv_score = np.sqrt(mean_squared_log_error(train_df['Calories'], oof_combined))

print(f"{'='*50}")
print(f"FINAL PIPELINE RESULTS")
print(f"{'='*50}")
print(f"Linear Regression only: 0.208762 (from exp_005)")
print(f"Linear + NN:             0.201961 (from exp_006)")
print(f"Linear + NN + XGBoost:   {cv_score:.6f}")
print(f"Improvement from NN:     {0.208762 - 0.201961:.6f}")
print(f"Improvement from XGB:    {0.201961 - cv_score:.6f}")
print(f"Total improvement:       {0.208762 - cv_score:.6f}")
print(f"\nBaseline XGBoost:        0.020470 (from exp_000)")
print(f"Pipeline vs Baseline:    {cv_score - 0.020470:.6f}")

# Check if we beat the baseline
if cv_score < 0.02047:
    print(f"\n✅ SUCCESS: Pipeline BEATS baseline by {0.02047 - cv_score:.6f}")
else:
    print(f"\n❌ Pipeline does NOT beat baseline (difference: {cv_score - 0.02047:.6f})")

FINAL PIPELINE RESULTS
Linear Regression only: 0.208762 (from exp_005)
Linear + NN:             0.201961 (from exp_006)
Linear + NN + XGBoost:   0.208070
Improvement from NN:     0.006801
Improvement from XGB:    -0.006109
Total improvement:       0.000692

Baseline XGBoost:        0.020470 (from exp_000)
Pipeline vs Baseline:    0.187600

❌ Pipeline does NOT beat baseline (difference: 0.187600)


## Save Predictions

In [11]:
# Save OOF predictions
oof_df = pd.DataFrame({
    'id': train_df['id'],
    'Calories_actual': train_df['Calories'],
    'Calories_pred': oof_combined,
    'lr_pred': lr_predictions,
    'nn_residual_pred': nn_residual_predictions,
    'xgb_residual_pred': oof_predictions
})

oof_df.to_csv('/home/code/experiments/007_xgboost_on_nn_residuals/oof_007_xgboost_residuals.csv', index=False)
print(f"OOF predictions saved. Shape: {oof_df.shape}")

# For test predictions, we need to combine all three stages
# Stage 1: Linear Regression (re-train on full data)
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder

# Prepare linear features (same as exp_005)
linear_features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
X_train_linear = train_df[linear_features].values
X_test_linear = test_df[linear_features].values

# One-hot encode Sex
sex_encoder = OneHotEncoder(sparse_output=False, drop='first')
sex_train = sex_encoder.fit_transform(train_df[['Sex']])
sex_test = sex_encoder.transform(test_df[['Sex']])

# Combine features
X_train_linear = np.hstack([X_train_linear, sex_train])
X_test_linear = np.hstack([X_test_linear, sex_test])

# Train linear model
linear_model = Ridge(alpha=1.0, random_state=SEED)
linear_model.fit(X_train_linear, train_df['Calories'])

# Predict linear component
linear_test_pred = linear_model.predict(X_test_linear)

# Stage 2: Neural Network (load test predictions from exp_006)
nn_test_pred = nn_oof['oof_prediction'].iloc[:len(test_df)].values

# Stage 3: XGBoost (use test_predictions from current experiment)
# Combine all three stages
test_final_pred = linear_test_pred + nn_test_pred + test_predictions

print(f"Linear test predictions shape: {linear_test_pred.shape}")
print(f"NN test predictions shape: {nn_test_pred.shape}")
print(f"XGBoost test predictions shape: {test_predictions.shape}")
print(f"Final test predictions shape: {test_final_pred.shape}")

OOF predictions saved. Shape: (8000, 6)
Linear test predictions shape: (2000,)
NN test predictions shape: (2000,)
XGBoost test predictions shape: (2000,)
Final test predictions shape: (2000,)


## Generate Submission

In [12]:
# Create submission file
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'Calories': test_final_pred
})

# Ensure proper format and clipping
submission_df['Calories'] = submission_df['Calories'].clip(
    lower=train_df['Calories'].min(), 
    upper=train_df['Calories'].max()
)

# Save submission
submission_path = '/home/submission/submission_007_xgboost_residuals.csv'
submission_df.to_csv(submission_path, index=False)

print(f"Submission saved to: {submission_path}")
print(f"Submission shape: {submission_df.shape}")
print(f"\nSubmission statistics:")
print(submission_df['Calories'].describe())

# Show first few rows
print(f"\nFirst 5 rows:")
print(submission_df.head())

Submission saved to: /home/submission/submission_007_xgboost_residuals.csv
Submission shape: (2000, 2)

Submission statistics:
count    2000.000000
mean      283.904177
std        95.035448
min       130.576972
25%       208.720153
50%       262.912489
75%       342.793989
max       500.000000
Name: Calories, dtype: float64

First 5 rows:
   id    Calories
0   0  231.286335
1   1  498.945765
2   2  451.410100
3   3  283.451680
4   4  279.757727
